# Index

1. [Imports](#Imports)
    1. [General](#Imports)
    2. [Distributed utilities](#Distributed)
    3. [Custom functions](#Custom-functions)
2. [Set Data path](#Set-data-path)
3. [Tune segmentation parameters](#Tune-segmentation-parameters)
4. [Set dask client](#Set-dask-client)
5. [Run extraction](#Run-extraction)
6. [Save properties](#Save-properties)

# Imports

In [ ]:
%pylab inline

import cv2

import numpy as np

import glob

import skimage as sk
import skimage.morphology
from skimage.filters import threshold_otsu, threshold_niblack
from skimage import measure
from skimage import io

import holoviews as hv
from holoviews.operation.datashader import datashade, shade
from colorcet import fire, gray
hv.extension('bokeh')

import mahotas as mh

import scipy.ndimage.morphology as morph

import pandas as pd

# Distributed utilities

In [ ]:
import dask
from dask.distributed import Client
from dask_jobqueue import SLURMCluster


In [ ]:
import distributed

In [ ]:
'Cluster' in distributed.deploy.__dict__.keys()

# Custom functions

In [ ]:
from apseg_utils import *

def extract_unlabeled(indx):
    
    img = sk.io.imread(phase_channel_files[indx])
    f_files = [x[indx] for x in fl_files]
    conn_comp, cell_masks, img_mask, maxima = extract_connected_components_phase(img, cell_masks=detect_rough_cells(img),min_cell_size=10, max_perc_contrast=None, cut_from_bottom=0,above_cell_pad=0, flip_cells=False,return_all=True,**kwargs)
    data = extract_cells(img, conn_comp, f_files)
    df = pd.DataFrame.from_dict(data)
    
    return df

# Set data path

In [ ]:
base_dir = '../images/e_03/'

In [ ]:
phase_channel_dir = base_dir + 'C1/'
fl_1_channel_dir = base_dir + 'C2/'
fl_2_channel_dir = base_dir + 'C3/'

phase_channel_files = glob.glob(phase_channel_dir + "*.tif")
phase_channel_files.sort()

fl_1_channel_files = glob.glob(fl_1_channel_dir + "*.tif")
fl_1_channel_files.sort()

fl_2_channel_files = glob.glob(fl_2_channel_dir + "*.tif")
fl_2_channel_files.sort()

fl_files = [fl_1_channel_files, fl_2_channel_files] # and so on

# Tune segmentation parameters

In [ ]:
init_niblack_k_def = -4#-4#-4#-0.3
maxima_niblack_k_def = -4#-4#-.75 #-0.75
init_smooth_sigma_def = 5#5#1.9#1.1
maxima_smooth_sigma_def = 9#9 #5# 2
maxima_niblack_window_size_def = 5#5#7# 5
init_niblack_window_size_def = 9#9#19# 9

kwargs = {"init_niblack_k" : init_niblack_k_def,
          "maxima_niblack_k" : maxima_niblack_k_def,
          "init_smooth_sigma" : init_smooth_sigma_def,
          "maxima_smooth_sigma" : maxima_smooth_sigma_def,
          "maxima_niblack_window_size" : maxima_niblack_window_size_def,
          "init_niblack_window_size" : init_niblack_window_size_def}

img = sk.io.imread(phase_channel_files[0])
conn_comp = extract_connected_components_phase(img,
                                               cell_masks=detect_rough_cells(img),
                                               min_cell_size=100, #15
                                               max_perc_contrast=None,
                                               cut_from_bottom=0,
                                               above_cell_pad=0, 
                                               flip_cells=False,#flips along y-axis
                                               return_all=False,
                                               init_niblack_k = init_niblack_k_def,
                                               maxima_niblack_k = maxima_niblack_k_def,
                                               init_smooth_sigma = init_smooth_sigma_def,
                                               maxima_smooth_sigma = maxima_smooth_sigma_def,
                                               maxima_niblack_window_size = maxima_niblack_window_size_def,
                                               init_niblack_window_size = init_niblack_window_size_def)

kwargs = {"init_niblack_k" : init_niblack_k_def,
          "maxima_niblack_k" : maxima_niblack_k_def,
          "init_smooth_sigma" : init_smooth_sigma_def,
          "maxima_smooth_sigma" : maxima_smooth_sigma_def,
          "maxima_niblack_window_size" : maxima_niblack_window_size_def,
          "init_niblack_window_size" : init_niblack_window_size_def}

holoplot(img, conn_comp)

In [ ]:
hv.Image(conn_comp)


In [ ]:
im1 = conn_comp.copy()
im1[im1<.4]=0
plt.imshow(im1)
plt.imshow(img)

# Set dask client

In [18]:
run_local = True
n_workers_init = 4

if run_local:
    client = Client()
    client.cluster.scale(n_workers_init)
    time.sleep(5) # allow client time to respond to scaling before display
else:
    # note the specifed walltime, don't use too much or too little, 01:30:00 is a good baseline, 
    # you just need enough time to finish 'gathering' to props_all before the jobs die
    # you can always spin up more jobs later
    # you will launch many jobs, so you don't need multiple processes, a lot of ram or multiple threads
    cluster = SLURMCluster(queue="short",walltime='01:30:00',cores=1,memory='6G',processes=1)
    cluster.start_workers(n_workers_init)
    client = Client(cluster)

client

TypeError: The dask.set_options function has been deprecated.
Please use dask.config.set instead

  Before: with dask.set_options(foo='bar'):
              ...
  After:  with dask.config.set(foo='bar'):
              ...

# Run extraction

In [33]:
fut = client.map(extract_unlabeled,range(len(phase_channel_files)))
dask.distributed.progress(fut)

VBox()

In [ ]:
len(fl_files[0])

# Save properties

In [35]:
props = client.gather(fut)
props_all = pd.concat(props)
#int_dtype_list = props_all.select_dtypes(include=np.int64).dtypes.index.tolist()
#props_all[int_dtype_list] = props_all[int_dtype_list].astype(np.uint16)
save_dir = base_dir+'seg_output'#
props_all['kind'] = 'unknown'
props_all.to_pickle(save_dir + '/Data_01.pkl')
print('Done!')

Done!
